## Import thư viện

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%config InlineBackend.figure_format = 'retina'
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
raw_path = os.getenv('RAW_PATH')
cleaned_path = os.getenv('CLEANED_PATH')

## Import csv

In [2]:
df_parent_categories = pd.read_csv(f'{raw_path}fahasha_parent_categories.csv')
df_child_categories = pd.read_csv(f'{raw_path}fahasha_child_categories.csv')
df_products = pd.read_csv(f'{raw_path}fahasha_products.csv')
df_comments = pd.read_csv(f'{raw_path}fahasha_comments.csv')

Hàm tóm tắt thông tin có trong dữ liệu

In [3]:
def summary_data(df):
    print(f'Dataframe có tất cả: {df.shape[0]} hàng và {df.shape[1]} cột\n\n')
    print(f'Kiểu dữ liệu:\n\n{df.dtypes} \n\n')
    print(f'Missing values:\n\n{df.isnull().sum()} \n\n')
    print('Duplicated rows:', df.duplicated().sum())
summary_data(df_products)

Dataframe có tất cả: 1772 hàng và 12 cột


Kiểu dữ liệu:

Liên kết                object
Mã sản phẩm             object
Tên sản phẩm            object
Mã danh mục              int64
Giá                     object
Giá Thị Trường          object
Số sản phẩm đã bán      object
Nhà xuất bản            object
Tác giả                 object
Số trang                object
Đánh Giá trung bình    float64
Số lượt đánh giá         int64
dtype: object 


Missing values:

Liên kết                 0
Mã sản phẩm              0
Tên sản phẩm             0
Mã danh mục              0
Giá                      0
Giá Thị Trường          56
Số sản phẩm đã bán     301
Nhà xuất bản             7
Tác giả                 10
Số trang                90
Đánh Giá trung bình      0
Số lượt đánh giá         0
dtype: int64 


Duplicated rows: 81


## Preprocessing


In [4]:
# tạo 1 bản sao của df để tiến hành xử lý dữ liệu
products = df_products.copy()

### Tác Giả

In [5]:
products[products['Tác giả'].isnull()]

,Liên kết,Mã sản phẩm,Tên sản phẩm,Mã danh mục,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá
201,https://www.fahasa.com/frieren-phap-su-tien-ta...,frieren12,Frieren - Pháp Sư Tiễn Táng - Tập 12 - Tặng Kè...,6718,45.0,NaN,1.1k,Kim Đồng,NaN,192.0,0.0,0
202,https://www.fahasa.com/frieren-phap-su-tien-ta...,frieren12db,Frieren - Pháp Sư Tiễn Táng - Tập 12 - Bản Đặc...,6718,110.0,NaN,1.1k,Kim Đồng,NaN,192.0,0.0,0
743,https://www.fahasa.com/tu-dien-y-hoc-dorland-a...,8935073096761,Từ Điển Y Học DorLand Anh - Việt,19,1.235.000,1.300.000,63,Y Học,NaN,1980.0,4.8,4
992,https://www.fahasa.com/speak-now-1-sb-pk-richa...,9780194030151,Speak Now 1 Sb Pk Richards & Bohl,5421,248.9,262.0,188,Oxford,NaN,NaN,4.0,3
1035,https://www.fahasa.com/start-with-why.html?fhs...,9780241958223,Start With Why,5004,270.9,301.0,98,Penguin Books UK,NaN,NaN,5.0,2
1098,https://www.fahasa.com/make-your-bed.html?fhs_...,9780718188863,Make Your Bed,4199,279.0,310.0,80,Penguin Books UK,NaN,144.0,0.0,0
1111,https://www.fahasa.com/be-the-hands-and-feet.h...,9780735291850,Be The Hands And Feet,4199,260.1,289.0,NaN,NXB Random house,NaN,256.0,0.0,0
1292,https://www.fahasa.com/wonderfully-wicked-witc...,9781472352378,Wonderfully Wicked Witches,5492,95.4,106.0,NaN,Parragon,NaN,NaN,0.0,0
1642,https://www.fahasa.com/the-ultimate-selfie-kit...,9781472393678,The Ultimate Selfie Kit,3497,163.800,182.000,NaN,Parragon Book Service Ltd,NaN,NaN,0.0,0
1698,https://www.fahasa.com/lullaby-music-box.html?...,9781760451882,Lullaby Music Box,5889,245.700,273.000,NaN,Lake Press,NaN,24,0.0,0


### Mã sản phẩm

In [6]:
# Kiểm tra các record không có mã sản phẩm
products[products['Mã sản phẩm'].isna()]

,Liên kết,Mã sản phẩm,Tên sản phẩm,Mã danh mục,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá


### Nhà xuất bản

In [7]:
# Đếm số lượng từng nhà xuất bản
products['Nhà xuất bản'].value_counts()

Nhà xuất bản
Dân Trí                  71
Thế Giới                 56
Hồng Đức                 51
Kim Đồng                 49
NXB Tổng Hợp TPHCM       43
                         ..
Disney Editions           1
Hachette Book             1
Black Dog & Leventhal     1
Fireside                  1
Spruce Books              1
Name: count, Length: 392, dtype: int64

1 vài nhà xuất bản bị thừa cụm NXB ở đầu. Ví dụ NXB Kim Đồng và Kim Đồng là giống nhau.

Replace các NXB bị thừa cụm NXB

In [8]:
products['Nhà xuất bản'] = products[ 'Nhà xuất bản'].replace(r'^NXB','', regex=True).str.strip()

In [9]:
products['Nhà xuất bản'].value_counts()

Nhà xuất bản
Hồng Đức        91
Dân Trí         89
Thế Giới        81
Kim Đồng        69
Thanh Niên      62
                ..
Familius         1
‎Ace             1
Đông A           1
Fireside         1
Spruce Books     1
Name: count, Length: 370, dtype: int64

Số lượng nhà xuất bản đã giảm xuống từ 392 còn 370.

### Giá Thị Trường

In [10]:
#Kiểm tra các record không có giá   
# Gán index là các record có giá thị trường = 'Không có giá'
non_old_price = products['Giá Thị Trường'] == 'Không có giá'
products[non_old_price].head(5)

,Liên kết,Mã sản phẩm,Tên sản phẩm,Mã danh mục,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá


Các dòng không có giá thị trường ta sẽ mặc định là chính giá bán.
Fill các sp không có giá trị trường bằng giá bán

In [11]:
# Slicing các record có giá thị trường = 'Không có giá' và gán giá trị mới
products.loc[non_old_price , 'Giá Thị Trường'] = products['Giá']

Kiểm tra kết quả

In [12]:
products[non_old_price].head(5)

,Liên kết,Mã sản phẩm,Tên sản phẩm,Mã danh mục,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá


### Lấy tên danh mục từ df_child_categories

In [13]:
# merge products với df_categories dựa vào cột 'Mã danh mục' và phương thức inner
# inner để giữ lại các record có mã danh mục trong cả 2 bảng
products = pd.merge(products, df_child_categories, on='Mã danh mục', how='inner')

In [14]:
#productsframe mới sau khi merge có thêm cột Tên danh mục
products[['Tên sản phẩm', 'Nhà xuất bản' , 'Tên danh mục c1' , 'Tên danh mục c2']].head(5)

,Tên sản phẩm,Nhà xuất bản,Tên danh mục c1,Tên danh mục c2
0,Búp Sen Xanh (Tái Bản 2020),Kim Đồng,Sách Trong Nước,Thiếu nhi
1,Lược Sử Nước Việt Bằng Tranh (Tái Bản 2022),Kim Đồng,Sách Trong Nước,Thiếu nhi
2,100 Kỹ Năng Sinh Tồn,Thanh Niên,Sách Trong Nước,Thiếu nhi
3,Tuổi Thơ Dữ Dội - Tập 1 (Tái Bản 2019),Kim Đồng,Sách Trong Nước,Thiếu nhi
4,Tuổi Thơ Dữ Dội - Tập 2 (Tái Bản 2019),Kim Đồng,Sách Trong Nước,Thiếu nhi


### Drop các cột không dùng để phân tích

In [15]:
col_drop = ['Liên kết_x' , 'Mã danh mục' , 'Liên kết_y']
products.drop(col_drop, axis=1, inplace=True)

### Xử lý 2 cột giá

In [16]:
products[['Giá' , 'Giá Thị Trường']].head(5)

,Giá,Giá Thị Trường
0,61.2,72.0
1,119.0,140.0
2,74.25,99.0
3,60.0,80.0
4,60.0,80.0


In [17]:
non_old_price = products['Giá Thị Trường'].isna()
products[non_old_price].head(5)

,Mã sản phẩm,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá,Tên danh mục c1,Tên danh mục c2
45,3300000026817,Sách Giáo Khoa Bộ Lớp 1 - Chân Trời Sáng Tạo -...,153.0,NaN,1.5k,Đại Học Sư Phạm TPHCM,Nhiều Tác Giả,NaN,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
46,9786040393371,Ngữ Văn 12 - Tập 1 (Chân Trời) (Chuẩn),23.0,NaN,1.5k,Giáo Dục Việt Nam,Nhiều Tác Giả,172.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
47,9786040351951,Toán 11 - Tập 1 (Chân Trời Sáng Tạo) (Chuẩn),21.0,NaN,1.2k,Giáo Dục Việt Nam,Nhiều Tác Giả,152.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
48,9786040393708,Global Success - Tiếng Anh 12 - Sách Học Sinh ...,70.0,NaN,1.2k,Giáo Dục Việt Nam,"Hoàng Văn Vân, Vũ Hải Hà, Chu Quang Bình, Hoàn...",155.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
49,9786044863061,Toán 12 - Tập 1 (Cánh Diều) (Chuẩn),16.0,NaN,1.2k,Đại Học Sư Phạm,Nhiều Tác Giả,126.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo


Fill các row không có giá thị trường thành giá hiện tại

In [18]:
products.loc[non_old_price , 'Giá Thị Trường'] = products['Giá']

Kết quả


In [19]:
products[non_old_price].head(5)

,Mã sản phẩm,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá,Tên danh mục c1,Tên danh mục c2
45,3300000026817,Sách Giáo Khoa Bộ Lớp 1 - Chân Trời Sáng Tạo -...,153.0,153.0,1.5k,Đại Học Sư Phạm TPHCM,Nhiều Tác Giả,NaN,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
46,9786040393371,Ngữ Văn 12 - Tập 1 (Chân Trời) (Chuẩn),23.0,23.0,1.5k,Giáo Dục Việt Nam,Nhiều Tác Giả,172.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
47,9786040351951,Toán 11 - Tập 1 (Chân Trời Sáng Tạo) (Chuẩn),21.0,21.0,1.2k,Giáo Dục Việt Nam,Nhiều Tác Giả,152.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
48,9786040393708,Global Success - Tiếng Anh 12 - Sách Học Sinh ...,70.0,70.0,1.2k,Giáo Dục Việt Nam,"Hoàng Văn Vân, Vũ Hải Hà, Chu Quang Bình, Hoàn...",155.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo
49,9786044863061,Toán 12 - Tập 1 (Cánh Diều) (Chuẩn),16.0,16.0,1.2k,Đại Học Sư Phạm,Nhiều Tác Giả,126.0,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo


Ta sẽ loại bỏ dấu chấm để chuyển cột về dạng số nguyên.

In [20]:
# Hàm nhân giá trị của cột 'Giá' và 'Giá Thị Trường' với 100
def convert_price(x):
        return x * 100

In [21]:
# replace các dấu chấm thành rỗng
products['Giá'] = products['Giá'].replace(r'\.' , '', regex=True)
products['Giá Thị Trường'] = products['Giá Thị Trường'].replace(r'\.' , '', regex=True)

# Chuyển kiểu dữ liệu về int
products['Giá Thị Trường'] = products['Giá Thị Trường'].astype(int)
products['Giá'] = products['Giá'].astype(int)

#apply hàm convert_price cho cột 'Giá' và 'Giá Thị Trường'

products['Giá'] = products['Giá'].apply(convert_price)
products['Giá Thị Trường'] = products['Giá Thị Trường'].apply(convert_price)

Kiểm tra kết quả

In [22]:
products[['Giá' , 'Giá Thị Trường']].head(5)

,Giá,Giá Thị Trường
0,61200,72000
1,119000,140000
2,742500,99000
3,60000,80000
4,60000,80000


In [23]:
products[['Giá' , 'Giá Thị Trường']].dtypes

Giá               int64
Giá Thị Trường    int64
dtype: object

### Xử lý cột số sản phẩm đã bán

In [24]:
sold_1k = products['Số sản phẩm đã bán'] == '1.0k'
products[sold_1k][['Tên sản phẩm','Giá','Giá Thị Trường','Số sản phẩm đã bán']].head(2)

,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán
15,Lược Sử Nước Việt Bằng Tranh - Viet Nam - A Br...,166600,196000,1.0k
52,Ngữ Văn 12 - Tập 2 (Chân Trời) (Chuẩn),19000,19000,1.0k


In [25]:
sold_10k_plus =products['Số sản phẩm đã bán'] == '10k+'
products[sold_10k_plus][['Tên sản phẩm','Giá','Giá Thị Trường','Số sản phẩm đã bán']].head(2)


,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán
0,Búp Sen Xanh (Tái Bản 2020),61200,72000,10k+
311,Chia Sẻ Từ Trái Tim (Thích Pháp Hòa),126000,168000,10k+


Cột sp đã bán có chứa 'k' được hiểu là 1000, ta sẽ xử lý nó về dạng số nguyên
10k+ được hiểu là trên 10 nghìn cuốn sách. do không có số liệu cụ thể nên ta cũng sẽ nhân cho 1000 để có số liệu tương đối.

In [26]:
non_sol = products['Số sản phẩm đã bán'].isna()
products[non_sol][['Số sản phẩm đã bán', 'Đánh Giá trung bình']].sample(10)

,Số sản phẩm đã bán,Đánh Giá trung bình
1114,NaN,0.0
1682,NaN,0.0
1395,NaN,0.0
1685,NaN,0.0
1460,NaN,0.0
1246,NaN,4.0
1495,NaN,0.0
1192,NaN,0.0
1280,NaN,0.0
1696,NaN,5.0


Các row có sp đã bán bị nan có vẻ cũng không có đánh giá trung bình nên ta sẽ replace nan bằng 0

In [27]:
products['Số sản phẩm đã bán'] = products['Số sản phẩm đã bán'].fillna(0)

Kết quả


In [28]:
products[non_sol][['Số sản phẩm đã bán', 'Đánh Giá trung bình']].sample(10)

,Số sản phẩm đã bán,Đánh Giá trung bình
1206,0,0.0
1482,0,0.0
1524,0,0.0
1560,0,0.0
1711,0,0.0
1297,0,0.0
1700,0,5.0
1190,0,0.0
1723,0,0.0
1698,0,0.0


In [29]:
# hàm chuyển đổi giá trị số sản phẩm đã bán
def convert_abbreviations(value):
    if isinstance(value, str):
        value = value.lower().strip()
        # Xử lý các giá trị kèm theo ký tự
        if value.endswith('k'):
            # Lấy từ ký tự đầu tới ký tự áp chót
            return float(value[:-1]) * 1000
        elif value.endswith('k+'):
            # Lấy từ ký tự đầu tới trước 2 ký tự cuối
            return float(value[:-2]) * 1000
        else:
            return float(value)
    return value

# Áp dụng hàm cho cột Số sản phẩm đã bán
products['Số sản phẩm đã bán'] = products['Số sản phẩm đã bán'].apply(convert_abbreviations)
# Chuyển kiểu dữ liệu của cột thành kiểu số nguyên
products['Số sản phẩm đã bán'] = products['Số sản phẩm đã bán'].astype(int)

Kiểm tra kết quả sau khi chuyển đổi


In [30]:
products[sold_1k][['Tên sản phẩm','Giá','Giá Thị Trường','Số sản phẩm đã bán']].head(2)

,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán
15,Lược Sử Nước Việt Bằng Tranh - Viet Nam - A Br...,166600,196000,1000
52,Ngữ Văn 12 - Tập 2 (Chân Trời) (Chuẩn),19000,19000,1000


In [31]:
products[sold_10k_plus][['Tên sản phẩm','Giá','Giá Thị Trường','Số sản phẩm đã bán']].head(2)

,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán
0,Búp Sen Xanh (Tái Bản 2020),61200,72000,10000
311,Chia Sẻ Từ Trái Tim (Thích Pháp Hòa),126000,168000,10000


### Features Engineering

#### Tạo cluster cho cột sản phẩm đã bán
Định nghĩa nhóm : 
 - từ 0 đến 500 : 0
 - từ 500 đến 999 : 1
 - từ 1000 đến 4999 : 2
 - từ 5000 đến 1000 : 3

In [32]:
def categorize_quantity(value):
    if 0 <= value < 500:
        return 0
    elif 500 <= value < 1000:
        return 1
    elif 1000 <= value < 5000:
        return 2
    elif 5000 <= value:
        return 3
    else:
        return None


products['Loại doanh số sản phẩm'] = products['Số sản phẩm đã bán'].apply(categorize_quantity)
products['Loại doanh số sản phẩm'] = products['Loại doanh số sản phẩm'].astype('category')

Kết quả cột vừa tạo

In [33]:
products[products['Loại doanh số sản phẩm'] == 0][['Tên sản phẩm' , 'Giá' ,'Số sản phẩm đã bán', 'Loại doanh số sản phẩm']].head(2)

,Tên sản phẩm,Giá,Số sản phẩm đã bán,Loại doanh số sản phẩm
34,Mẹ Hỏi Bé Trả Lời 3-4 Tuổi (Tái Bản 2019),25500,499,0
35,Hoàng Tử Bé (Tái Bản),60000,485,0


In [34]:
products[products['Loại doanh số sản phẩm'] == 1][['Tên sản phẩm' , 'Giá' ,'Số sản phẩm đã bán', 'Loại doanh số sản phẩm']].head(2)


,Tên sản phẩm,Giá,Số sản phẩm đã bán,Loại doanh số sản phẩm
16,Đất Rừng Phương Nam (Tái Bản),607500,998,1
17,Những Câu Chuyện Tò Mò Của Bé - Con Có Thể Đán...,21000,986,1


In [35]:
products[products['Loại doanh số sản phẩm'] == 2][['Tên sản phẩm' , 'Giá' ,'Số sản phẩm đã bán', 'Loại doanh số sản phẩm']].head(2)


,Tên sản phẩm,Giá,Số sản phẩm đã bán,Loại doanh số sản phẩm
2,100 Kỹ Năng Sinh Tồn,742500,2400,2
3,Tuổi Thơ Dữ Dội - Tập 1 (Tái Bản 2019),60000,2000,2


In [36]:
products[products['Loại doanh số sản phẩm'] == 3][['Tên sản phẩm' , 'Giá' ,'Số sản phẩm đã bán', 'Loại doanh số sản phẩm']].head(2)


,Tên sản phẩm,Giá,Số sản phẩm đã bán,Loại doanh số sản phẩm
0,Búp Sen Xanh (Tái Bản 2020),61200,10000,3
1,Lược Sử Nước Việt Bằng Tranh (Tái Bản 2022),119000,8000,3


#### Tạo cột mới hiển thị phần trăm giảm

In [37]:
# products['Phần trăm giảm'] = round((products['Giá Thị Trường'] - products['Giá']) / products['Giá Thị Trường'], 2)

Kết quả 

In [38]:
# products[['Giá Thị Trường' , 'Giá', 'Phần trăm giảm']].head(10)

### Xoá hàng bị trùng

Ví dụ 1 dòng dữ liệu bị trùng

In [39]:
products[products['Mã sản phẩm'] == '3300000027210']

,Mã sản phẩm,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá,Tên danh mục c1,Tên danh mục c2,Loại doanh số sản phẩm
65,3300000027210,Sách Giáo Khoa Bộ Lớp 6 - Chân Trời Sáng Tạo -...,187000,187000,824,Giáo Dục Việt Nam,Bộ Giáo Dục Và Đào Tạo,NaN,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo,1
69,3300000027210,Sách Giáo Khoa Bộ Lớp 6 - Chân Trời Sáng Tạo -...,187000,187000,824,Giáo Dục Việt Nam,Bộ Giáo Dục Và Đào Tạo,NaN,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo,1


Tổng các dòng bị trùng

In [40]:
print(products.duplicated().sum())

81


Tiến hành xoá dòng bị trùng, giữ lại dòng đầu tiên

In [41]:
# Xóa các dòng trùng lặp, drop_duplicates() sẽ giữ lại dòng đầu tiên
# Nếu muốn giữ lại dòng cuối cùng, thêm tham số keep='last'
products = products.drop_duplicates()

Kết quả

In [42]:
products.duplicated().sum()

np.int64(0)

Sau khi drop thì dòng bị trùng trước đó chỉ còn 1 dòng

In [43]:
products[products['Mã sản phẩm']=='3300000027210']

,Mã sản phẩm,Tên sản phẩm,Giá,Giá Thị Trường,Số sản phẩm đã bán,Nhà xuất bản,Tác giả,Số trang,Đánh Giá trung bình,Số lượt đánh giá,Tên danh mục c1,Tên danh mục c2,Loại doanh số sản phẩm
65,3300000027210,Sách Giáo Khoa Bộ Lớp 6 - Chân Trời Sáng Tạo -...,187000,187000,824,Giáo Dục Việt Nam,Bộ Giáo Dục Và Đào Tạo,NaN,0.0,0,Sách Trong Nước,Giáo khoa - Tham khảo,1


Xuất ra file csv các file đã clean

In [44]:
clean_path = Path(cleaned_path).mkdir(parents=True, exist_ok=True)
products.to_csv(cleaned_path + 'products_cleaned.csv', index=False, encoding='utf-8-sig' )
df_child_categories.to_csv(cleaned_path + 'child_categories_cleaned.csv', index=False, encoding='utf-8-sig')
df_parent_categories.to_csv(cleaned_path + 'parent_categories_cleaned.csv', index=False, encoding='utf-8-sig')
df_comments.to_csv(cleaned_path + 'comments_cleaned.csv', index=False, encoding='utf-8-sig')